In [497]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [498]:
# Initialize df for team data
teams_df = pd.DataFrame(columns=['Team', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS'])
# Populate Team Data with web scraping
def get_team_data():
    teams = []
    W = []
    L = []
    T = []
    W_L = []
    PF = []
    PA = []
    PD = []
    MoV = []
    SoS = []
    SRS = []
    OSRS = []
    DSRS = []
    
    html_text = requests.get('https://www.pro-football-reference.com/years/2022/index.htm').text
    soup = BeautifulSoup(html_text, 'lxml')
    tables = soup.find_all('tbody')
    
    for table in tables:
        teams_data = table.find_all('tr', class_="")
        for team_data in teams_data:
            teams.append(team_data.find('a').text)
            W.append(team_data.find_all('td')[0].text)
            L.append(team_data.find_all('td')[1].text)
            T.append(team_data.find_all('td')[2].text)
            W_L.append(team_data.find_all('td')[3].text)
            PF.append(team_data.find_all('td')[4].text)
            PA.append(team_data.find_all('td')[5].text)
            PD.append(team_data.find_all('td')[6].text)
            MoV.append(team_data.find_all('td')[7].text)
            SoS.append(team_data.find_all('td')[8].text)
            SRS.append(team_data.find_all('td')[9].text)
            OSRS.append(team_data.find_all('td')[10].text)
            DSRS.append(team_data.find_all('td')[11].text)
    teams_df['Team'] = teams
    teams_df['W'] = W
    teams_df['L'] = L
    teams_df['T'] = T
    teams_df['W-L%'] = W_L
    teams_df['PF'] = PF
    teams_df['PA'] = PA
    teams_df['PD'] = PD
    teams_df['MoV'] = MoV
    teams_df['SoS'] = SoS
    teams_df['SRS'] = SRS
    teams_df['OSRS'] = OSRS
    teams_df['DSRS'] = DSRS
get_team_data()

In [499]:
# Sort teams alphabetically for easy merging
teams_df_sort = teams_df.sort_values(by = 'Team').reset_index(drop=True)

In [500]:
# Initialize df for offense redzone data
off_red_df = pd.DataFrame(columns=['Team', 'off_red_per'])
# Populate offense redzone data with web scraping
def get_off_red_data():
    teams = []
    percent = []
    html_text = requests.get('https://www.teamrankings.com/nfl/stat/red-zone-scoring-pct?date=2022-11-10').text
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('tbody')
    for row in table.find_all('tr'):
        teams.append(row.find('a').text)
        percent.append(row.find_all('td')[2].text)
    off_red_df['Team'] = teams
    off_red_df['off_red_per'] = percent
get_off_red_data()

In [501]:
# Sort teams alphabetically for easy merging
off_red_df_sort = off_red_df.sort_values(by = 'Team').reset_index(drop=True)

In [502]:
# Create new team data df with offense redzone data
new_teams_df = pd.concat([teams_df_sort, off_red_df_sort['off_red_per']], axis=1)
new_teams_df['off_red_per'] = new_teams_df['off_red_per'].str.rstrip('%').astype('float') / 100.0

In [503]:
# Initialize Schedule df for 2022
schedule_df = pd.read_csv('spreadspoke_scores.csv').groupby('schedule_season').get_group(2022)
pd.set_option('display.max_columns', None)

In [504]:
# Merge team data with schedule data
df = schedule_df.join(new_teams_df.set_index('Team'), on='team_home').join(new_teams_df.set_index('Team'), on='team_away', rsuffix='_away')
pd.set_option('display.max_columns', None)

In [505]:
# Visualize data
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,off_red_per,W_away,L_away,T_away,W-L%_away,PF_away,PA_away,PD_away,MoV_away,SoS_away,SRS_away,OSRS_away,DSRS_away,off_red_per_away
13232,9/8/2022,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,52,SoFi Stadium,False,72.0,0.0,NaN,DOME,3,5,0,.375,131,173,-42,-5.3,1.0,-4.3,-4.0,-0.3,0.5000,6,2,0,.750,220,118,102,12.8,1.6,14.3,6.2,8.2,0.5357
13233,9/11/2022,2022,1,False,Arizona Cardinals,21.0,44.0,Kansas City Chiefs,KC,-6.0,54,State Farm Stadium,False,NaN,NaN,NaN,NaN,3,6,0,.333,203,241,-38,-4.2,0.1,-4.1,-1.0,-3.1,0.5769,6,2,0,.750,243,189,54,6.8,-0.6,6.1,9.3,-3.2,0.7353
13234,9/11/2022,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,44,Mercedes-Benz Stadium,False,NaN,NaN,NaN,NaN,4,6,0,.400,232,250,-18,-1.8,-1.4,-3.2,0.4,-3.6,0.6207,3,6,0,.333,212,227,-15,-1.7,-0.6,-2.3,0.7,-3.0,0.5714
13235,9/11/2022,2022,1,False,Carolina Panthers,24.0,26.0,Cleveland Browns,CAR,-1.0,42,Bank of America Stadium,False,NaN,NaN,NaN,NaN,3,7,0,.300,204,243,-39,-3.9,-1.2,-5.1,-1.9,-3.2,0.5455,3,5,0,.375,200,199,1,0.1,0.2,0.3,3.0,-2.7,0.6129
13236,9/11/2022,2022,1,False,Chicago Bears,19.0,10.0,San Francisco 49ers,SF,-6.5,38,Soldier Field,False,NaN,NaN,NaN,NaN,3,6,0,.333,187,216,-29,-3.2,0.7,-2.5,0.7,-3.2,0.5517,4,4,0,.500,176,147,29,3.6,-1.7,2.0,-1.2,3.2,0.5769


In [506]:
# Get completed games
comp_games = df[df['score_home'].notna()]
# Get uncompleted games
uncomp_games = df[df['score_home'].isna()]

In [507]:
# Determine winner and loser for completed games
for index, row in comp_games.iterrows():
    if comp_games.loc[index, 'score_home'] > comp_games.loc[index, 'score_away']:
        comp_games.loc[index, 'Home_Winner'] = 1
    else:
        comp_games.loc[index, 'Home_Winner'] = 0
    # else:
    #     comp_games.at[index, 'Winner'] = "T"

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_9552/3030477943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_games.loc[index, 'Home_Winner'] = 0


In [508]:
schedule_df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13232,9/8/2022,2022,1,False,Los Angeles Rams,10.0,31.0,Buffalo Bills,BUF,-2.0,52,SoFi Stadium,False,72.0,0.0,NaN,DOME
13233,9/11/2022,2022,1,False,Arizona Cardinals,21.0,44.0,Kansas City Chiefs,KC,-6.0,54,State Farm Stadium,False,NaN,NaN,NaN,NaN
13234,9/11/2022,2022,1,False,Atlanta Falcons,26.0,27.0,New Orleans Saints,NO,-5.5,44,Mercedes-Benz Stadium,False,NaN,NaN,NaN,NaN
13235,9/11/2022,2022,1,False,Carolina Panthers,24.0,26.0,Cleveland Browns,CAR,-1.0,42,Bank of America Stadium,False,NaN,NaN,NaN,NaN
13236,9/11/2022,2022,1,False,Chicago Bears,19.0,10.0,San Francisco 49ers,SF,-6.5,38,Soldier Field,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13499,1/8/2023,2022,18,False,Philadelphia Eagles,NaN,NaN,New York Giants,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN
13500,1/8/2023,2022,18,False,Pittsburgh Steelers,NaN,NaN,Cleveland Browns,NaN,NaN,NaN,Acrisure Stadium,False,NaN,NaN,NaN,NaN
13501,1/8/2023,2022,18,False,San Francisco 49ers,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN
13502,1/8/2023,2022,18,False,Seattle Seahawks,NaN,NaN,Los Angeles Rams,NaN,NaN,NaN,Lumen Field,False,NaN,NaN,NaN,NaN


In [509]:
# Remove unnecessary columns
comp_games.drop(schedule_df.iloc[0:0, 0:4] + schedule_df.iloc[0:0, 5:7] + schedule_df.iloc[0:0, 8:], axis=1, inplace=True)
uncomp_games.drop(schedule_df.iloc[0:0, 0:3] + schedule_df.iloc[0:0, 5:6] + schedule_df.iloc[0:0, 8:], axis=1, inplace=True)

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_9552/210060255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_games.drop(schedule_df.iloc[0:0, 0:4] + schedule_df.iloc[0:0, 5:7] + schedule_df.iloc[0:0, 8:], axis=1, inplace=True)
/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_9552/210060255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uncomp_games.drop(schedule_df.iloc[0:0, 0:3] + schedule_df.iloc[0:0, 5:6] + schedule_df.iloc[0:0, 8:], axis=1, inplace=True)


In [510]:
# Display training data
comp_games.head()

,team_home,team_away,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,off_red_per,W_away,L_away,T_away,W-L%_away,PF_away,PA_away,PD_away,MoV_away,SoS_away,SRS_away,OSRS_away,DSRS_away,off_red_per_away,Home_Winner
13232,Los Angeles Rams,Buffalo Bills,3,5,0,.375,131,173,-42,-5.3,1.0,-4.3,-4.0,-0.3,0.5000,6,2,0,.750,220,118,102,12.8,1.6,14.3,6.2,8.2,0.5357,0.0
13233,Arizona Cardinals,Kansas City Chiefs,3,6,0,.333,203,241,-38,-4.2,0.1,-4.1,-1.0,-3.1,0.5769,6,2,0,.750,243,189,54,6.8,-0.6,6.1,9.3,-3.2,0.7353,0.0
13234,Atlanta Falcons,New Orleans Saints,4,6,0,.400,232,250,-18,-1.8,-1.4,-3.2,0.4,-3.6,0.6207,3,6,0,.333,212,227,-15,-1.7,-0.6,-2.3,0.7,-3.0,0.5714,0.0
13235,Carolina Panthers,Cleveland Browns,3,7,0,.300,204,243,-39,-3.9,-1.2,-5.1,-1.9,-3.2,0.5455,3,5,0,.375,200,199,1,0.1,0.2,0.3,3.0,-2.7,0.6129,0.0
13236,Chicago Bears,San Francisco 49ers,3,6,0,.333,187,216,-29,-3.2,0.7,-2.5,0.7,-3.2,0.5517,4,4,0,.500,176,147,29,3.6,-1.7,2.0,-1.2,3.2,0.5769,1.0


In [511]:
# Set X and Y
X = comp_games.drop(['Home_Winner'], 1)
y = comp_games['Home_Winner']

# Scale data for more accurate predictions
from sklearn.preprocessing import scale
cols = [comp_games.columns[2:-1]]
for col in cols:
    X[col] = scale(X[col])

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_9552/3230569309.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = comp_games.drop(['Home_Winner'], 1)


In [512]:
# Columnize any strings in dataset
def preprocess_features(X):
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
        output = output.join(col_data)
    return output

X = preprocess_features(X)

/var/folders/zm/hl17sd8j21bf4w5xf5lnrj0w0000gn/T/ipykernel_9552/614488492.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, col_data in X.iteritems():


In [513]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [520]:
# Initialize algorithms
LR_clf = LogisticRegression()
SVC_clf = SVC()
xgb_clf = xgb.XGBClassifier(seed = 1)

# Fit algorithms to training data
LR_clf.fit(X_train, y_train)
SVC_clf.fit(X_train, y_train)
xgb_clf.fit(X_train, y_train)

# Predict using algorithms
LR_pred = LR_clf.predict(X_test)
SVC_pred = SVC_clf.predict(X_test)
XGB_pred = xgb_clf.predict(X_test)

# Cross validate algorithms
LR_score = cross_val_score(LR_clf, X, y)
SVC_score = cross_val_score(SVC_clf, X, y)
XGB_score = cross_val_score(xgb_clf, X, y)

In [521]:
# Print accuracy and score of algorithms
print(f'Linear Regression:', '\nAccuracy:', accuracy_score(y_test, LR_pred), '\nF1 Score: ', f1_score(y_test, LR_pred), '\nCross Validation Score:', LR_score.mean())
print()
print(f'SVC:', '\nAccuracy:', accuracy_score(y_test, SVC_pred), '\nF1 Score: ', f1_score(y_test, SVC_pred), '\nCross Validation Score: ', SVC_score.mean())
print()
print(f'XGBoost:', '\nAccuracy:', accuracy_score(y_test, XGB_pred), '\nF1 Score: ', f1_score(y_test, XGB_pred), '\nCross Validation Score: ', XGB_score.mean())

Linear Regression: 
Accuracy: 0.6923076923076923 
F1 Score:  0.7142857142857143 
Cross Validation Score: 0.5373333333333334

SVC: 
Accuracy: 0.8461538461538461 
F1 Score:  0.8571428571428571 
Cross Validation Score:  0.676

XGBoost: 
Accuracy: 0.7692307692307693 
F1 Score:  0.7999999999999999 
Cross Validation Score:  0.5686666666666667
